In [1]:
import numpy as np
import scipy as sp

In [2]:
#Define dimensions and boundary conditions
L_y = 10 #rows
L_x = 10 #columns
isPeriodicVert = False
isPeriodicHor = False

In [3]:
def get_nth_dim(array, nth_dim, index): #index values on the nth dimension
    '''Credit to https://stackoverflow.com/a/45462833'''
    return array[(slice(None), )*nth_dim + (index, )]

In [4]:
def hop_1D(L,isPeriodic): #hopping in 1D with given boundary condition
    #L: number of lattice points
    #(clearly) based on your code 

    lattice = np.arange(L) #1D lattice

    cos = sp.sparse.dok_matrix((L,L), dtype=complex) #initialize matrices as sparse dok matrices
    sin = sp.sparse.dok_matrix((L,L), dtype=complex)

    for i in range(L):
        nextPos = lattice[(i+1)%L]
        doHop = isPeriodic or nextPos != 0
        if doHop:
            cos[i,nextPos] = 1/2
            sin[i,nextPos] = 1j/2 #j^2 = -1

    #we need not compute negative dir, just add hermitian adjoint
    cos += cos.conj().T
    sin += sin.conj().T

    return cos.toarray(), sin.toarray()

In [5]:
def hop_2D(L_y,L_x,isPerY,isPerX): #find cos, sin matrices for hopping in the dir of each axis; specify if either axis is periodic.
    #L_y: vertical size (n rows)
    #L_x: horizontal size (n cols)
    #isPerY,X: is periodic (boudnary cond) in respective direction?
    n_points = L_y*L_x
    lattice = np.arange(n_points).reshape((L_y,L_x))


    cosY = sp.sparse.dok_matrix((n_points,n_points), dtype=complex) #initialize matrices as sparse dok matrices
    sinY = sp.sparse.dok_matrix((n_points,n_points), dtype=complex)
    cosX = sp.sparse.dok_matrix((n_points,n_points), dtype=complex) 
    sinX = sp.sparse.dok_matrix((n_points,n_points), dtype=complex)

    for y in range(L_y):
        for x in range(L_x):

            y_nhbr = lattice[(y-1)%L_y, x] #next neighbor in positive vert dir (up)
            doHopY = isPerY or y_nhbr != 0 #only hop if periodic or next point is not wrapped 
            if doHopY:
                cosY[lattice[y,x], y_nhbr] = 1/2
                sinY[lattice[y,x], y_nhbr] = 1j/2


            x_nhbr = lattice[y, (x+1)%L_x] #next neighbor in positive hor dir (right)
            doHopX = isPerX or x_nhbr != 0
            if doHopX:
                cosX[lattice[y,x], x_nhbr] = 1/2
                sinX[lattice[y,x], x_nhbr] = 1j/2

    ver_list = [cosY, sinY]
    hor_list = [cosX, sinX]

    sup_list = [ver_list, hor_list] #idea to generalize to N dimensions

    for i in sup_list: #for each axis dir list 
        for j in i: #for each cos, sin in a given direction
            j += j.conj().T  #add hermitian adjoint (as opposed to also computing negative hop direction)

    for i in range(len(sup_list)):
        for j in range(len(sup_list[i])):
            sup_list[i][j] = sup_list[i][j].toarray() #convert to numpy array

    return sup_list

In [6]:
def init_cos_sin(n_points): 
    cos = sp.sparse.dok_matrix((n_points,n_points), dtype=complex) #initialize matrices as sparse dok matrices
    sin = sp.sparse.dok_matrix((n_points,n_points), dtype=complex)
    return cos, sin

In [7]:
def hop_ndim(shape,isPer):
    #shape: a tuple describing the size each dimension
    #isPer: a list of boolean with the same length as shape; is periodic in given dir?

    n_dims = len(shape)
    n_points = 1 #initalize
    for i in shape: #product of all dimension size
        n_points *= i

    lattice = np.arange(n_points).reshape(shape)

    super_list = [None]*n_dims #list of all cos, sin arrays

    for i in range(n_dims):
        super_list[i] = [init_cos_sin(n_points)]

In [21]:
def iter_for_loop(shape, n_dims): #iterate on all points of n_dims
    if (n_dims > 1):
        iter_for_loop(shape, n_dims-1)
        for i in range(shape[n_dims-1]):
            print(i,end=' ')
        print()

